In [2]:
import sys 
sys.path.append('../../..')
from main_multitask_multimodal import LLM_MultitaskMultimodal

In [3]:
import json

In [79]:
model = LLM_MultitaskMultimodal.load_from_checkpoint('../../../models/product_query_textonly_multitask/version_2/epoch=0-step=800.ckpt'
                                                     ).cuda().eval()

INFO:root:Unused kwargs when getting hkunlp/instructor-base: {'distance_func': 'cosine', 'loss_type': 'cross-entropy', 'margin': None, 'hidden_states_type': 'encoder-mean', 'add_simcse': False, 'manual_loss_type': 'manual_mse', 'auto_task_weight': False, 'multitask_specs_dict': {'emb_singlemodal_wishquery2googletitle': None, 'seqclf_singlemodal_wishtitle2v121tax': {'specs': {'clf_head': 'clf_head_product', 'clf_weight_type': None, 'label_map_file': '/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt', 'label_type': 'multilabel_taxonomy'}}, 'seqclf_singlemodal_wishquery2v121tax': {'specs': {'clf_head': 'clf_head_query', 'clf_weight_type': None, 'label_map_file': '/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths_withunknown.txt', 'label_type': 'multilabel_taxonomy'}}}, 'head_dict': {'clf_head_product': {'purpose': 'seqclf', 'type': 'linear', 'in_features': 768, 'out_features': 6037}, 'clf_head_query': {'purpose':

# qualitative check

In [80]:
inputs = model.tokenizer([
        "Represent the E-commerce product title for retrieval: Big Dildo",
        "Represent the E-commerce product title for retrieval: Milk",
        "Represent the E-commerce product title for retrieval: Iphone", 
        "Represent the E-commerce product title for retrieval: Samsung"
    ], return_tensors='pt', padding=True, truncation=True, max_length=50).to('cuda')

In [81]:
embs_products = model.get_hidden_states(**inputs)

In [82]:
inputs = model.tokenizer([
        "Represent the E-commerce search query for retrieval: sex toy",
        "Represent the E-commerce search query for retrieval: food",
        "Represent the E-commerce search query for retrieval: electronics", 
        "Represent the E-commerce search query for retrieval: electronics apple",
        "Represent the E-commerce search query for retrieval: electronics galaxy"
    ], return_tensors='pt', padding=True, truncation=True, max_length=50).to('cuda')

In [83]:
embs_query = model.get_hidden_states(**inputs)

In [84]:
from torch import nn

In [85]:
embs_query = nn.functional.normalize(embs_query)
embs_products = nn.functional.normalize(embs_products)

In [88]:
(-embs_query.mm(embs_products.T).detach().cpu().numpy()).argsort(1)

array([[0, 1, 2, 3],
       [1, 3, 2, 0],
       [3, 2, 1, 0],
       [2, 3, 1, 0],
       [3, 2, 1, 0]])

In [90]:
from InstructorEmbedding import INSTRUCTOR
model2 = INSTRUCTOR('hkunlp/instructor-base')
sentence = "3D ActionSLAM: wearable person tracking in multi-floor environments"
instruction = "Represent the E-commerce product title for retrieval:"
instruction2 = "Represent the E-commerce search query for retrieval:"
embs_products = model2.encode([[instruction,sentence] for sentence in [
    'Big Dildo', 'Milk', 'Iphone', 'Samsung'
]])
embs_query = model2.encode([[instruction,sentence] for sentence in [
    'sex toy', 'food', 'electronics', 'electronics apple', 'electronics galaxy'
]])

INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: hkunlp/instructor-base


load INSTRUCTOR_Transformer
max_seq_length  512


INFO:sentence_transformers.SentenceTransformer:Use pytorch device: cuda


In [91]:
(-1 * embs_query.dot(embs_products.T)).argsort(1)

array([[0, 3, 2, 1],
       [1, 3, 0, 2],
       [3, 2, 0, 1],
       [2, 3, 0, 1],
       [3, 2, 0, 1]])

In [92]:
model2

INSTRUCTOR(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: T5EncoderModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False})
  (2): Dense({'in_features': 768, 'out_features': 768, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (3): Normalize()
)

# convert

In [93]:
from torch import nn

In [94]:
from transformers import T5EncoderModel

In [105]:
import torch

In [132]:
class T5TextEncoder(T5EncoderModel):
    def forward(self,
            input_ids = None,
            attention_mask = None,
            head_mask = None,
            inputs_embeds = None,
            output_attentions = None,
            output_hidden_states = None,
            return_dict = None):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        encoder_outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            inputs_embeds=inputs_embeds,
            head_mask=head_mask,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )
        token_embeddings = encoder_outputs.last_hidden_state
        input_mask_expanded = attention_mask.unsqueeze(-1).float()
        sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
        sum_mask = input_mask_expanded.sum(1)
        return sum_embeddings / sum_mask

In [133]:
model_hf = T5TextEncoder.from_pretrained('hkunlp/instructor-base')

In [ ]:
model_hf.load_state_dict(model.transformer.state_dict(), strict=False)

In [135]:
inputs = model.tokenizer([
        "Represent the E-commerce search query for retrieval: sex toy",
        "Represent the E-commerce search query for retrieval: food",
        "Represent the E-commerce search query for retrieval: electronics", 
        "Represent the E-commerce search query for retrieval: electronics apple",
        "Represent the E-commerce search query for retrieval: electronics galaxy"
    ], return_tensors='pt', padding=True, truncation=True, max_length=50).to('cuda')

In [136]:
model_hf.eval().cuda()
model_hf(**inputs)

tensor([[ 0.1049,  1.1187, -0.0632,  ...,  0.3223,  0.1130, -0.2071],
        [ 1.7254,  1.9041,  1.4327,  ..., -1.2619,  0.1200,  0.6428],
        [-0.3199, -0.7741, -0.0882,  ..., -1.7333,  1.0926, -0.1274],
        [-1.1760, -0.3870, -0.4076,  ..., -1.7242,  1.3544,  0.7890],
        [ 0.1942,  1.4429, -1.4297,  ...,  0.4009,  0.1020, -1.1584]],
       device='cuda:0', grad_fn=<DivBackward0>)

In [137]:
model.eval().cuda()
model.get_hidden_states(**inputs)

tensor([[ 0.1049,  1.1187, -0.0632,  ...,  0.3223,  0.1130, -0.2071],
        [ 1.7254,  1.9041,  1.4327,  ..., -1.2619,  0.1200,  0.6428],
        [-0.3199, -0.7741, -0.0882,  ..., -1.7333,  1.0926, -0.1274],
        [-1.1760, -0.3870, -0.4076,  ..., -1.7242,  1.3544,  0.7890],
        [ 0.1942,  1.4429, -1.4297,  ...,  0.4009,  0.1020, -1.1584]],
       device='cuda:0', grad_fn=<DivBackward0>)

In [138]:
input_ids = inputs['input_ids']
attention_mask = inputs['attention_mask']

In [141]:
ONNX_PATH = f"onnx_ckpt/model.onnx"
torch.onnx.export(
    model_hf,
    (input_ids, attention_mask),
    ONNX_PATH,
    input_names=["input_ids", "attention_mask"],
    output_names=["embs"],
    dynamic_axes={
        'input_ids': {0: 'batch_size', 1: 'sequence_length'},
        'attention_mask': {0: 'batch_size', 1: 'sequence_length'},
        'embs': {0: 'batch_size'}
    },
    opset_version=13,
    do_constant_folding=True
)

In [143]:
import onnxruntime as ort

# Instantiate the ONNX Runtime session
ort_session = ort.InferenceSession(ONNX_PATH)

outputs = ort_session.run(
    None,
    {
        "input_ids": input_ids.detach().cpu().numpy(),
        "attention_mask": attention_mask.detach().cpu().numpy(),
    }
)

In [144]:
outputs

[array([[ 0.10488483,  1.1187394 , -0.06322645, ...,  0.3222735 ,
          0.11304621, -0.2071243 ],
        [ 1.725398  ,  1.9040842 ,  1.4327086 , ..., -1.2619073 ,
          0.11998194,  0.64281094],
        [-0.31993556, -0.7741432 , -0.08823031, ..., -1.7333367 ,
          1.0925678 , -0.12737033],
        [-1.175967  , -0.38696894, -0.40760002, ..., -1.7242362 ,
          1.35442   ,  0.7889614 ],
        [ 0.19422801,  1.4429021 , -1.4296671 , ...,  0.40093282,
          0.10199822, -1.1584119 ]], dtype=float32)]